In [20]:
import glob
import os
import re
import subprocess
import shutil
import yaml
from pathlib import Path

In [5]:
m = yaml.safe_load(open('/data/pecgs/test_sample/C3L-00677/summary.yaml'))
m

{'id': 'C3L-00677',
 'rna-seq': {'tumor': {'R1': {'filepath': '/data/pecgs/test_sample/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R1_001.fastq.gz'},
   'R2': {'filepath': '/data/pecgs/test_sample/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R2_001.fastq.gz'}}},
 'wxs': {'normal': {'barcode': 'CPT0071960002',
   'filepath': '/data/pecgs/test_sample/C3L-00677/wxs/5cee818c-61e8-4441-bd9a-5c984cde9c6b_gdc_realn.bam'},
  'tumor': {'barcode': 'CPT0071100006',
   'filepath': '/data/pecgs/test_sample/C3L-00677/wxs/1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn.bam'}}}

###### top

In [25]:
sample = m['id']
fq_1 = m['rna-seq']['tumor']['R1']['filepath']
fq_2 = m['rna-seq']['tumor']['R2']['filepath']
cpu = 16

In [56]:
genome_lib_dir = '/data/pecgs/fusion/STAR-Fusion_dependencies/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/'
genome_db = '/data/pecgs/fusion/ericscript_dependencies/ericscript_db_homosapiens_ensembl84/'
# ericscript = '/data/pecgs/fusion/ericscript-0.5.5/ericscript.pl'
bwts = '/data/pecgs/fusion/Integrate_dependencies/bwts/'
integrate_fasta = '/data/pecgs/fusion/Integrate_dependencies/STAR/hg38.fa'
integrate_annotations = '/data/pecgs/fusion/Integrate_dependencies/annot.ensembl.GRCh38.txt'
combine_call_script = '/data/pecgs/fusion/Fusion_hg38_scripts/combine_call.pl'
filter_script = '/data/pecgs/fusion/Fusion_hg38_scripts/filter.pl'


###### step 1: star fusion

In [8]:
# mkdir -p STAR_FUSION
# STAR-Fusion --left_fq $fq_1 --right_fq $fq_2 --CPU $cpu --examine_coding_effect -O STAR_FUSION --genome_lib_dir $genome_lib_dir --verbose_level 2 1> logs/STAR-Fusion.out 2> logs/STAR-Fusion.err



In [13]:
fusion_out = 'STAR_FUSION'


In [28]:
Path(fusion_out).mkdir(parents=True, exist_ok=True)
cmd = f'STAR-Fusion --left_fq {fq_1} --right_fq {fq_2} --CPU {cpu} --examine_coding_effect -O {fusion_out} --genome_lib_dir {genome_lib_dir} --verbose_level 2'
cmd



'STAR-Fusion --left_fq /data/pecgs/test_sample/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R1_001.fastq.gz --right_fq /data/pecgs/test_sample/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R2_001.fastq.gz --CPU 16 --examine_coding_effect -O STAR_FUSION --genome_lib_dir /data/pecgs/fusion/STAR-Fusion_dependencies/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/ --verbose_level 2'

In [15]:
# Cleanup big files from STAR-Fusion

#rm STAR_FUSION/Aligned.out.bam
#rm -r STAR_FUSION/star-fusion.preliminary
#rm -r STAR_FUSION/_*
#gzip STAR_FUSION/Chimeric.out.junction

In [ ]:
if os.path.exists(f'{fusion_out}/Aligned.out.bam'):
    os.remove(f'{fusion_out}/Aligned.out.bam')
    
dirs = glob.glob(f'{fusion_out}/_*')
for d in dirs:
    shutil.rmtree(d)
    
if os.path.exists(f'{fusion_out}/star-fusion.preliminary'):
    shutil.rmtree(f'{fusion_out}/star-fusion.preliminary')


###### step 2: ericscript

In [53]:
ericscript_out = 'ERICSCRIPT'
ericscript = 'ericscript.pl'

In [54]:
# /storage1/fs1/dinglab/Active/Projects/PECGS/PECGS_pipeline/Fusion/ericscript-0.5.5/ericscript.pl -o ERICSCRIPT --remove -ntrim 0 --refid homo_sapiens -db $genome_db -p $cpu -name $sample $fq_1 $fq_2



In [57]:
cmd = f'{ericscript} -o {ericscript_out} --remove -ntrim 0 --refid homo_sapiens -db {genome_db} -p {cpu} -name {sample} {fq_1} {fq_2}'
cmd



'/data/pecgs/fusion/ericscript-0.5.5/ericscript.pl -o ERICSCRIPT --remove -ntrim 0 --refid homo_sapiens -db /data/pecgs/fusion/ericscript_dependencies/ericscript_db_homosapiens_ensembl84/ -p 16 -name C3L-00677 /data/pecgs/test_sample/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R1_001.fastq.gz /data/pecgs/test_sample/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R2_001.fastq.gz'

###### step 3: STAR

In [32]:
star_out = 'STAR'
star_genome_dir = os.path.join(genome_lib_dir, 'ref_genome.fa.star.idx')

In [30]:
# STAR --runThreadN 80 --genomeDir /storage1/fs1/dinglab/Active/Projects/PECGS/PECGS_pipeline/Fusion/STAR-Fusion_dependencies/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/ref_genome.fa.star.idx --readFilesCommand zcat --readFilesIn ../${fq_1} ../${fq_2} --outSAMtype BAM Unsorted --outFileNamePrefix star --chimSegmentMin 18 --chimOutType WithinBAM --outSAMunmapped Within 1> ../logs/star.out 2> ../logs/star.err



In [33]:
Path(star_out).mkdir(parents=True, exist_ok=True)
cmd = f'STAR --runThreadN 80 --genomeDir {star_genome_dir} --readFilesCommand zcat --readFilesIn {fq_1} {fq_2} --outSAMtype BAM Unsorted --outFileNamePrefix {star_out}/ --chimSegmentMin 18 --chimOutType WithinBAM --outSAMunmapped Within'
cmd


'STAR --runThreadN 80 --genomeDir /data/pecgs/fusion/STAR-Fusion_dependencies/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/ref_genome.fa.star.idx --readFilesCommand zcat --readFilesIn /data/pecgs/test_sample/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R1_001.fastq.gz /data/pecgs/test_sample/C3L-00677/rna-seq/170906_UNC32-K00270_0058_BHLJ7FBBXX_TGACCA_S4_L007_R2_001.fastq.gz --outSAMtype BAM Unsorted --outFileNamePrefix STAR/ --chimSegmentMin 18 --chimOutType WithinBAM --outSAMunmapped Within'

In [34]:
cmd = f'samtools sort {star_out}/Aligned.out.bam {star_out}/Aligned.out.sorted'
cmd

'samtools sort STAR/Aligned.out.bam STAR/Aligned.out.sorted'

In [35]:
cmd = f'samtools index {star_out}/Aligned.out.sorted.bam'
cmd

'samtools index STAR/Aligned.out.sorted.bam'

###### step 4: Integrate

In [37]:
integrate_out = 'INTEGRATE'

In [38]:
# Integrate fusion -reads INTEGRATE/reads.txt -sum INTEGRATE/summary.tsv -ex INTEGRATE/exons.tsv -bk INTEGRATE/breakpoints.tsv -vcf INTEGRATE/bk_sv.vcf -bedpe INTEGRATE/fusions.bedpe $fasta $annot $bwts$bam_dir/starAligned.out.sorted.bam $bam_dir/starAligned.out.sorted.bam 1> logs/Integrate.out 2> logs/Integrate.err



In [39]:
Path(integrate_out).mkdir(parents=True, exist_ok=True)
cmd = f'Integrate fusion -reads {integrate_out}/reads.txt -sum {integrate_out}/summary.tsv -ex {integrate_out}/exons.tsv -bk {integrate_out}/breakpoints.tsv -vcf {integrate_out}/bk_sv.vcf -bedpe {integrate_out}/fusions.bedpe {integrate_fasta} {integrate_annotations} {bwts} {star_out}/Aligned.out.sorted.bam {star_out}/Aligned.out.sorted.bam'
cmd


'Integrate fusion -reads INTEGRATE/reads.txt -sum INTEGRATE/summary.tsv -ex INTEGRATE/exons.tsv -bk INTEGRATE/breakpoints.tsv -vcf INTEGRATE/bk_sv.vcf -bedpe INTEGRATE/fusions.bedpe /data/pecgs/fusion/Integrate_dependencies/STAR/hg38.fa /data/pecgs/fusion/Integrate_dependencies/annot.ensembl.GRCh38.txt /data/pecgs/fusion/Integrate_dependencies/bwts/ STAR/starAligned.out.sorted.bam STAR/starAligned.out.sorted.bam'

In [ ]:
for fp in [f'{star_out}/Aligned.out.bam',
           f'{star_out}/Aligned.out.sorted.bam',
           f'{star_out}/Aligned.out.sorted.bam.bai']:
    if os.path.exists(fp):
        os.remove(fp)

for fp in [f'{star_out}/star_STARtmp']:
    if os.path.exists(fp):
        shutil.rmtree(fp)

###### step 5: merge fusions

In [41]:
merge_fusions_out = 'Merged_Fusions'

In [ ]:
# perl /storage1/fs1/dinglab/Active/Projects/PECGS/PECGS_pipeline/Fusion/Fusion_hg38_scripts/combine_call.pl $sample STAR_FUSION/star-fusion.fusion_predictions.abridged.coding_effect.tsv ERICSCRIPT/${sample}.results.total.tsv INTEGRATE/summary.tsv INTEGRATE/breakpoints.tsv Merged_Fusions 1> logs/Merge.out 2> logs/Merge.err


In [43]:
Path(merge_fusions_out).mkdir(parents=True, exist_ok=True)
cmd = f'perl {combine_call_script} {sample} {fusion_out}/star-fusion.fusion_predictions.abridged.coding_effect.tsv {ericscript_out}/{sample}.results.total.tsv {integrate_out}/summary.tsv {integrate_out}/breakpoints.tsv {merge_fusions_out}'
cmd


'perl /data/pecgs/fusion/Fusion_hg38_scripts/combine_call.pl C3L-00677 STAR_FUSION/star-fusion.fusion_predictions.abridged.coding_effect.tsv ERICSCRIPT/C3L-00677.results.total.tsv INTEGRATE/summary.tsv INTEGRATE/breakpoints.tsv Merged_Fusions'

In [48]:
cmd = f'perl {filter_script} {merge_fusions_out}'

cmd

'perl /data/pecgs/fusion/Fusion_hg38_scripts/filter.pl Merged_Fusions'